# Entrega Final
## Limpieza y preprocesado y creación de modelo Random Forest
Este notebook contiene operaciones de limpieza y prepocesado de datos, asi como la creacion del modelo de Random Forest.

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer, KNNImputer
from scipy.stats import zscore
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from google.colab import files
import os

In [ ]:
# Cargar json
files.upload()

In [ ]:
# Cargar competencia de Kaggle
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

In [ ]:
# descomprimir archivos
!unzip udea*.zip > /dev/null
!wc *.csv

In [ ]:
# Cargar archivo train.csv
file = "train.csv"
df = pd.read_csv(file)
df.head()

## Preprocesado de datos
### Imputación de Datos Faltantes

In [ ]:
# Separar las columnas numéricas y categóricas
columnas_numericas = df.select_dtypes(include=["float64", "int64"]).columns
columnas_categoricas = df.select_dtypes(include=["object"]).columns

# Imputar columnas numéricas usando KNNImputer
imputar_numericas = KNNImputer(n_neighbors=5)
df[columnas_numericas] = imputar_numericas.fit_transform(df[columnas_numericas])

# Imputar columnas categóricas con la moda (más frecuente)
imputar_categoricas = SimpleImputer(strategy="most_frequent")
df[columnas_categoricas] = imputar_categoricas.fit_transform(df[columnas_categoricas])

# Verificar si quedan valores faltantes
print("Valores faltantes después de la imputación:")
print(df.isnull().sum())

### Conversión de Variables Categóricas a One-Hot Encoding]

In [ ]:
# Codificación de variables categóricas
label_encoder = LabelEncoder()
df["RENDIMIENTO_GLOBAL"] = label_encoder.fit_transform(df["RENDIMIENTO_GLOBAL"])
for col in df.select_dtypes(include=["object"]).columns:
    df[col] = label_encoder.fit_transform(df[col])

### Escalado de Valores Numéricos



In [ ]:
# Escalado con StandardScaler
scaler = StandardScaler()
df[columnas_numericas] = scaler.fit_transform(df[columnas_numericas])

print("Variables numéricas escaladas:")
print(df[columnas_numericas].head())

### Detectar valores atípicos: con Z-score y reemplazarlos por la mediana


In [ ]:
# Definir umbral
umbral = 3

# Aplicar Z-score y reemplazar valores atípicos por la mediana
for col in columnas_numericas:
    if df[col].std() == 0:
        continue
    z_scores = zscore(df[col])
    mediana = df[col].median()
    df[col] = np.where(np.abs(z_scores) > umbral, mediana, df[col])

print("Valores atípicos suavizados reemplazando por la mediana usando Z-score.")

## Creación de modelo Random Forest
### 1. Dividir variables X y Y en entrenamiento y prueba

In [ ]:
# Separar variables predictoras y objetivo
X = df.drop(columns=["RENDIMIENTO_GLOBAL", "ID"])
y = df["RENDIMIENTO_GLOBAL"]

In [ ]:
# Dividir el conjunto de datos para validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### 2. Definición de modelo Random Forest

In [ ]:
# Creacion modelo Random Forest
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train, y_train)
y_pred_rf = modelo_rf.predict(X_val)
precision_rf = accuracy_score(y_val, y_pred_rf)

print("Accuracy en validación con Random Forest:", precision_rf)

### 3. Aplicar procedimiento a datos de prueba

In [ ]:
file = "test.csv"
df_test = pd.read_csv(file)

### 4. Imputar valores faltantes a df_test

In [ ]:
# Imputar valores faltantes en `df_test`
df_test[columnas_numericas] = imputar_numericas.transform(df_test[columnas_numericas])

### 5. Normalización de valores

In [ ]:
# Normalizar las columnas numéricas en `df_test`
df_test[columnas_numericas] = scaler.transform(df_test[columnas_numericas])

### 6. Predicción de los datos de prueba con el modelo creado

In [ ]:
# Predicción en los datos de prueba
X_test = df_test.drop(columns=["ID"])
predicciones = modelo_rf.predict(X_test)

### 7. Crear dataframe para guardar predicción

In [ ]:
df_submission = pd.DataFrame({"ID": df_test["ID"], "RENDIMIENTO_GLOBAL": predicciones})

### 8. Crear archivo .csv

In [ ]:
# Guardar el archivo de envío en formato CSV
df_submission.to_csv("submission.csv", index=False)
print("Archivo 'submission.csv' creado con éxito.")

In [ ]:
df_submission.shape

In [ ]:
!head submission.csv

### 9. Enviar archivo .csv a la competencia

In [ ]:
!kaggle competitions submit -c udea-ai4eng-20242 -f submission.csv -m "Manuela Sanchez Arenas modelo Random Forest"